# ***Living Earth*** Exploring DEA Land Cover <img align="right" src="../Supplementary_data/dea_logo.jpg">

* [**Sign up to the DEA Sandbox**](https://docs.dea.ga.gov.au/setup/sandbox.html) to run this notebook interactively from a browser
* **Compatibility:** Notebook currently compatible with the `DEA Sandbox` environment
* **Products used:** 
[ga_ls_landcover_class_cyear_2](https://explorer.sandbox.dea.ga.gov.au/ga_ls_landcover_class_cyear_2)


## Loading up an area based on a box

Through Geoscience Australia's (GA) Digital Earth Australia (DEA) Land Cover, and as part of *Living Earth*, land cover maps have been generated for 34 years (1988 to 2020) and for each year from environmental descriptors retrieved or classified entirely from Landsat sensor data and according to the Food and Agriculture Organisation (FAO) Land Cover Classification System (Version 2; https://www.fao.org/4/y7220e/y7220e00.htm).  The spatial resolution of the mapping is 25 m
(see DEA Land Cover) at https://cmi.ga.gov.au/data-products/dea/607/dea-land-cover-landsat)

This notebook allows you to explore the land cover maps for anywhere in Australia and for any period of time (noting the maps are generated annually).

You can define an area (based on coordinates of a box) and then you can load up and analyse the land cover and other environmental descriptors (e.g., lifeform, water persistence) used in their construction.  

To assist in the selection of your coordinates, you can click on the image (to be displayed in this notebook) and copy coordinates from the display.  Another option is to go to http://geojson.io and identify coordinates from there.  However, to get started, two areas have been pre-selected.

## Description 

This notebook will demonstrate how to load up an area of the Digital Earth Australia (DEA) land cover map and environmental descriptors used in their construction and description.   

The notebooks take you through the process of:

1. Setting up a time period for analysis.
2. Identifying an area based on a pre-defined coordinates for a rectangle.  
3. Displaying the Level 3 land cover maps and environmental descriptors of lifeform, canopy cover, water persistence and water seasonality.  

If you are new to DEA Land Cover, it is recommended you look at the introductory [DEA Land Cover notebook](../DEA_datasets/DEA_Land_Cover.ipynb) first. 

***

### Load packages
The cell below imports Python packages that are used for the analysis.
The first command is `%matplotlib inline`, which ensures figures plot correctly in the Jupyter notebook.
The following commands import various functionality: 

* `sys` provides access to helpful support functions in the `dea_plotting` module. 
* `datacube` provides the ability to query and load data.
* `matplotlib` provides the ability to format and manipulate plots.

Note that other libraries are listed but will be used in later iterations of this notebook.  

In [1]:
# Initial imports and setup
import sys
import datacube
import os, re
import matplotlib.pyplot as plt
import display_tools

import rioxarray
import warnings
import xarray as xr

from dea_tools.landcover import lc_colourmap, make_colorbar, plot_land_cover
from dea_tools.plotting import display_map, rgb, xr_animation # Added xr_animation RML

from datacube.testutils.io import rio_slurp_xarray
from datacube.utils.cog import write_cog
from datacube.utils.geometry import Geometry
from ipywidgets import Button, Dropdown, FloatText, GridspecLayout, Layout, HBox, VBox, Output, Tab, interact, widgets


from time import time
warnings.filterwarnings("ignore")
dc = datacube.Datacube()
time_range = None

from display_tools import display_da
from landcover import lc_colourmap, make_colorbar, plot_land_cover

# Ensure openpyxl is installed
!pip install openpyxl

added a new line here
another line


## Getting started

To run this analysis, run all the cells in the notebook starting with the 'Load packages and connect to the datacube' cell.

### Load packages

Load key Python packages and supporting functions for the analysis, then connect to the datacube. 

### Connect to the datacube
Connect to the datacube so we can access DEA data. 

In [2]:
dc = datacube.Datacube(app="Land_cover_change_mapping")

### Select the time interval
Select a time range between 1988 to 2020, noting that the processing may take a little longer when more years are included. 

In [3]:
# Set the range of dates for the analysis
time_range = ("2009", "2020")

### Select your area
Select your area of interest by listing the top left and top right latitude and longitude coordinates.  The coordinates of the two areas that have been preselected are provided and you should comment out (using a '#') the area that you DO NOT want to use. 

In [4]:
# Create the 'query' dictionary object, which contains the longitudes, latitudes and time range. 

#Hinchinbrook Island, Queensland, Australia**

#central_lat = -18.3711
#central_lon = 146.1693
#

#query = {
#    "x": (146.0593, 146.2793),
#    "y": (-18.4811, -18.2611),
#    "time": time_range,
#}

#Sydney, New South Wales, Australia**

query = {
    "x": (151.12, 151.34),
    "y": (-34.01, -33.79),
    "time": time_range,
}
display_map(x=query["x"], y=query["y"])

### Load DEA land cover
This section reads the data from the data cube, with this including the land cover and environmental descriptor measurements (ga_ls_landcover_class_cyear_2) and the output Coordinate Reference System (CRS; see https://epsg.io/?q= and also https://datacarpentry.org/organization-geospatial/03-crs.html)). Note that the area of interest is interrogated based on the 'query'.  

In [5]:
# Load DEA Land Cover data from the datacube


lc = dc.load(
    product="ga_ls_landcover_class_cyear_2",
    output_crs="EPSG:3577",
    measurements=[
        "level3",
        "lifeform",
        "vegetation_cover",
        "water_seasonality",
        "water_state",
        "intertidal",
        "water_persistence",
        "bare_gradation",
        "full_classification",
    ],
    resolution=(-25, 25),
    **query
)

Following the load step, printing the `lc` `xarray.Dataset` object will give you insight into all of the data that was loaded.
Do this by running the next cell.

The data are represented by the following aspects:
- `Dimensions`: the names of data dimensions, frequently `time`, `x` and `y`, and number of entries in each
- `Coordinates`: the coordinate values for each point in the data cube, including time.
- `Data variables`: the observations loaded, with this including the environmental descriptors and full land cover classification.
- `Attributes`: additional useful information about the data, such as the `crs` (coordinate reference system)

In [6]:
lc

<xarray.Dataset> Size: 112MB
Dimensions:              (time: 12, y: 1092, x: 948)
Coordinates:
  * time                 (time) datetime64[ns] 96B 2009-01-01 ... 2020-01-01
  * y                    (y) float64 9kB -3.819e+06 -3.819e+06 ... -3.846e+06
  * x                    (x) float64 8kB 1.751e+06 1.751e+06 ... 1.774e+06
    spatial_ref          int32 4B 3577
Data variables:
    level3               (time, y, x) uint8 12MB 112 112 112 112 ... 220 220 220
    lifeform             (time, y, x) uint8 12MB 1 1 1 1 1 1 1 ... 0 0 0 0 0 0 0
    vegetation_cover     (time, y, x) uint8 12MB 12 12 12 12 12 12 ... 0 0 0 0 0
    water_seasonality    (time, y, x) uint8 12MB 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0
    water_state          (time, y, x) uint8 12MB 0 0 0 0 0 0 0 ... 1 1 1 1 1 1 1
    intertidal           (time, y, x) uint8 12MB 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0
    water_persistence    (time, y, x) uint8 12MB 0 0 0 0 0 0 0 ... 1 1 1 1 1 1 1
    bare_gradation       (time, y, x) uint8 12MB 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0
    full_classification  (time, y, x) uint8 12MB 28 28 28 28 ... 101 101 101 101
Attributes:
    crs:           EPSG:3577
    grid_mapping:  spatial_ref

Note that you can interrogate each layer separately by typing 'lc.level3' or 'lc.water_persistence'.  You will see here the values (e.g., if using lc.lifeform, you will see values of 1 for woody or 2 for herbaceous vegetation).

In [7]:
lc.lifeform

<xarray.DataArray 'lifeform' (time: 12, y: 1092, x: 948)> Size: 12MB
array([[[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 0, ..., 0, 0, 0],
        [1, 0, 2, ..., 0, 0, 0],
        [1, 0, 2, ..., 0, 0, 0]],

       [[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 2, 2, ..., 0, 0, 0],
        [2, 2, 2, ..., 0, 0, 0],
        [0, 2, 2, ..., 0, 0, 0]],

       [[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
...
        ...,
        [1, 1, 0, ..., 0, 0, 0],
        [0, 0, 2, ..., 0, 0, 0],
        [1, 0, 2, ..., 0, 0, 0]],

       [[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 2, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        [1, 1, 0, ..., 0, 0, 0]],

       [[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [2, 1, 0, ..., 0, 0, 0],
        [1, 0, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]]], dtype=uint8)
Coordinates:
  * time         (time) datetime64[ns] 96B 2009-01-01 2010-01-01 ... 2020-01-01
  * y            (y) float64 9kB -3.819e+06 -3.819e+06 ... -3.846e+06 -3.846e+06
  * x            (x) float64 8kB 1.751e+06 1.751e+06 ... 1.774e+06 1.774e+06
    spatial_ref  int32 4B 3577
Attributes:
    units:         1
    nodata:        0
    crs:           EPSG:3577
    grid_mapping:  spatial_ref

### Displaying the land cover and environmental descriptors for the selected time period.
By running this section, you will be able to select checkbox widgets associated with each of the data layers.   Initially, tick "Full Classification" and then plot these selected data. Then, view the environmental descriptors used in the construction of the land cover maps for each year.

In [8]:
# Section 6: Checkbox widgets for selecting datasets
level3_checkbox = widgets.Checkbox(description='Level 3', value=False)
lifeform_checkbox = widgets.Checkbox(description='Lifeform', value=False)
vegetation_cover_checkbox = widgets.Checkbox(description='Vegetation Cover', value=False)
water_state_checkbox = widgets.Checkbox(description='Water State', value=False)
water_persistence_checkbox = widgets.Checkbox(description='Water Persistence', value=False)
full_classification_checkbox = widgets.Checkbox(description='Full Classification', value=False)

output = widgets.Output()

def plot_selected_datasets():
    with output:
        output.clear_output()
        selected_datasets = []
        if level3_checkbox.value:
            selected_datasets.append('level3')
        if lifeform_checkbox.value:
            selected_datasets.append('lifeform')
        if vegetation_cover_checkbox.value:
            selected_datasets.append('vegetation_cover')
        if water_state_checkbox.value:
            selected_datasets.append('water_state')
        if water_persistence_checkbox.value:
            selected_datasets.append('water_persistence')            
        if full_classification_checkbox.value:
            selected_datasets.append('full_classification')

        for dataset in selected_datasets:
            plt.figure()
            if dataset == 'level3':
                plot_land_cover(lc.level3)
            elif dataset == 'lifeform':
                plot_land_cover(lc.lifeform)
            elif dataset == 'vegetation_cover':
                plot_land_cover(lc.vegetation_cover)
            elif dataset == 'water_state':
                plot_land_cover(lc.water_state)
            elif dataset == 'water_persistence':
                plot_land_cover(lc.water_persistence)
            elif dataset == 'full_classification':
                plot_land_cover(lc.full_classification)
            plt.show()

plot_datasets_button = widgets.Button(description='Plot Selected Data')
plot_datasets_button.on_click(lambda b: plot_selected_datasets())

checkboxes_box = widgets.HBox([
    level3_checkbox, 
    lifeform_checkbox, 
    vegetation_cover_checkbox, 
    water_state_checkbox, 
    water_persistence_checkbox, 
    full_classification_checkbox,
])
display(checkboxes_box)
display(plot_datasets_button)
display(output)

Button(description='Plot Selected Data', style=ButtonStyle())

Output()

### Exploring DEA Land Cover
The land cover maps are constructed from environmental descriptors generated for each year, with these including those that are categorical (i.e., lifeform, water state) and continuous (i.e., vegetation cover (%), water persistence (time)).  Go ahead and explore Australia and see if you can observe any changes associated with, for example, vegetation burning, deforestation, flooding, drought or coastal erosion.   

## Tags
Browse all available tags on the DEA User Guide's [Tags Index](https://docs.dea.ga.gov.au/genindex.html)

**Tags**: :index:`sandbox compatible`, :index:`landsat 5`, :index:`landsat 7`, :index:`landsat 8`, 
:index:`DEA Land Cover`, :index:`time series`, :index: `LCCS`, :index:`colour maps`, :index:`data visualisation`